# Practical 8

## Introduction to Data Mining 

### Ensemble Learning

"Ensembles" are an approach for making predictions that involves combining several models together. 

The concept is illustrated below with a logistic regression model, a decision tree and a k nearest neighbours model. Instead of making a prediction with one of the models, predictions are made by taking input from all of the models. Specifically, given some features X, a combined prediction y  is found by first obtaining predictions from each of the models and combining it by some method: for example voting (taking the majority of class predictions) or taking the average (also applicable in regression). 

![title](images/ensembles.png)

The approach has significant benefits and many of the most effective algorithms in data mining are based on ensembles (for example Random Forests, Ada Boost). Benefits of ensembles includes that predictions from several models combined on average has a lower error than any of the individual models separately, the combined model also generally has less tendency for overfitting. 

This can be understood to be because the variance (i.e. of the predictions) is reduced and if the models used in constructing the combined model are independent (that is they are have different data and/or different modelling processes used in determining them) then the resulting model will have less "bias" than any of the indivudal models (e.g. linear models have a particular bias to finding linear patterns and combining linear and non linear models prevents this bias).

### Bagging and Boosting

Bagging, boosting and stacking are three variants or approaches for generating ensembles. They should be thought of as "Meta" algorithms ( algorithms that have as input or operate on several sub algorithms that themselves act on the data).

#### Bagging
Creating several models (which are then combined) by generating additional training data through random resampling from the original data set and then using different subsets of this data to different train models. See also http://scikit-learn.org/stable/modules/ensemble.html#bagging

#### Boosting
Boosting is a more complex approach that extends bagging by selectively choosing data to train models in the ensemble instead of randomly choosing the data. In boosting an initial set of models is generated from subsets of the training data, then additional models are trained by selecting training data that consists of examples that were most likely to be misclassified by the original models obtained in the first step. See also http://scikit-learn.org/stable/modules/ensemble.html

#### Stacking
In stacking several models are used rather than just varying the data. In addition, instead of using simple mathematical function such as the mean, or majority or weighted voting to combine predictions, a data mining model (such as logistic regression) can be used to combine the outputs/predictions of many indivudal models together.
